In [37]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [96]:
# df = pd.read_csv('D:\IET software-CC-CL\Commit Classification\experiment\Commit_dataset.csv', encoding="cp1252")

# label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}
label2id = {'Corrective':0,'Adaptive':1,'Perfective':2}
# df = df.replace({"3_labels": label2id})
# df = pd.read_csv(r'dataset.csv',engine="python")
# df['text'] = "['DIFF]" + df['diff']
# # df = df.replace({"2_labels": label2id})
# print(df)

train = pd.read_csv('train.csv',index_col=0)
# train = train.rename(columns={'3_labels':'label','comment':'text'})
train = train.replace({"label": label2id})
train.fillna(0, inplace=True)
test = pd.read_csv('test.csv',index_col=0)
# test = test.rename(columns={'3_labels':'label','comment':'text'})
test = test.replace({"label": label2id})
test.fillna(0, inplace=True)
print(len(test))



1758


In [97]:
from datasets import Dataset, load_metric

In [98]:
# train_dataset = Dataset.from_pandas(train)
# test_dataset = Dataset.from_pandas(test)

In [99]:
train

,commit_id,project,text,label,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Extract Variable,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder
0,0531b8bff5c14d9504beefb4ad47f473e3a22932,ReactiveX-RxJava,Change hasException to hasThrowable--,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1198,bb33f4f482c3914efb4b2498ec2d34f79beff145,tapiji,Removes icon folder from the tool's core plugi...,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
702,59d1720a5d1020d1167bea1a31b9616a685398a2,orientdb,Fixed bug on oidentifable but not odocument- f...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1012,2daa05f2bd185e3de5c2e85afc86b2bc6194b41b,Valadoc,gtkdoc: accept %<numeric>\n,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1189,39982c0bc8abc5b7bfa8731f6df65037727297f4,tapiji,Renames branch 'master_tmp' to 'master'.\n\n,2,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
501,6bc9677bbe919cd5bcadd8af2f5b0c838757a4ce,drools,[DROOLS-293] fix ObjectTypeNode id creation an...,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
899,ff7d4eebd8ebbf011656313dca8c6ee1a598c2aa,spring-framework,Polish AbstractHandlerMethodMapping---,2,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1758,c1c63c500a3d0737c081b3eb5616b17767d965e6,hunterhacker$jdom,Remove duplicate TextHelper functionality. Mov...,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1137,d698699901d72f73212e302ad715c97a4c4fc10e,tapiji,Adapts the namespace of all TapiJI plug-ins to...,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
595,dc9563874163393853426a050003de459a86f048,kotlin,Method cache for ClassDescriptorSerializer--,2,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
len(train)

35

In [102]:
train['label'].value_counts()

2    18
0    11
1     6
Name: label, dtype: int64

In [103]:
test['label'].value_counts()

0    592
1    584
2    582
Name: label, dtype: int64

In [104]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"accuracy": accuracy_score, "recall": recall_score, "f1":f1_score}

In [105]:
train

,commit_id,project,text,label,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Extract Variable,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder
0,0531b8bff5c14d9504beefb4ad47f473e3a22932,ReactiveX-RxJava,Change hasException to hasThrowable--,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1198,bb33f4f482c3914efb4b2498ec2d34f79beff145,tapiji,Removes icon folder from the tool's core plugi...,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
702,59d1720a5d1020d1167bea1a31b9616a685398a2,orientdb,Fixed bug on oidentifable but not odocument- f...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1012,2daa05f2bd185e3de5c2e85afc86b2bc6194b41b,Valadoc,gtkdoc: accept %<numeric>\n,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1189,39982c0bc8abc5b7bfa8731f6df65037727297f4,tapiji,Renames branch 'master_tmp' to 'master'.\n\n,2,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
501,6bc9677bbe919cd5bcadd8af2f5b0c838757a4ce,drools,[DROOLS-293] fix ObjectTypeNode id creation an...,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
899,ff7d4eebd8ebbf011656313dca8c6ee1a598c2aa,spring-framework,Polish AbstractHandlerMethodMapping---,2,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1758,c1c63c500a3d0737c081b3eb5616b17767d965e6,hunterhacker$jdom,Remove duplicate TextHelper functionality. Mov...,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1137,d698699901d72f73212e302ad715c97a4c4fc10e,tapiji,Adapts the namespace of all TapiJI plug-ins to...,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
595,dc9563874163393853426a050003de459a86f048,kotlin,Method cache for ClassDescriptorSerializer--,2,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
# id2label = {0: "Adaptive", 1: "POSITIVE"}
# label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [107]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased", num_labels=3
)
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [108]:
training_args = TrainingArguments(
    output_dir="./my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

In [109]:
encoded_train = tokenizer(train['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
encoded_test = tokenizer(test['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)

In [110]:
class CommitDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [111]:
train_dataset = CommitDataset(encoded_train, list(train['label']))
test_dataset = CommitDataset(encoded_test, list(test['label']))

In [112]:
def compute_metrics(eval_pred):
    accuracy = load_metric('accuracy')
    precision = load_metric("precision")
    recall = load_metric("recall")
    f1 = load_metric("f1")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy.compute(predictions=predictions, references=labels)
    precision = precision.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall.compute(predictions=predictions, references=labels,average='weighted')
    f1 = f1.compute(predictions=predictions, references=labels,average='weighted')
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [113]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

<ipython-input-110-ebcd56f8ad98>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.097959,{'precision': 0.31976249306205967},{'recall': 0.3310580204778157},{'f1': 0.27998034944290306},{'accuracy': 0.3310580204778157}


Trainer is attempting to log a value of "{'precision': 0.31976249306205967}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.3310580204778157}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.27998034944290306}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.3310580204778157}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=3, training_loss=1.1143261591593425, metrics={'train_runtime': 69.0401, 'train_samples_per_second': 0.507, 'train_steps_per_second': 0.043, 'total_flos': 9208969620480.0, 'train_loss': 1.1143261591593425, 'epoch': 1.0})